# WiDS 2022 Datathon
### Nancy Hamdan

# Installing Libraries

In [ ]:
!pip install category_encoders

# Importing Libraries

In [ ]:
import math
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold, KFold
import category_encoders as ce

from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

# Getting Data

In [4]:
tr = pd.read_csv('train.csv')
tt = pd.read_csv('test.csv')

sub = pd.read_csv('sample_solution.csv')
sub['id'] = tt['id']

del tr['id']
del tt['id']

In [ ]:
tr.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30F,days_below_20F,days_below_10F,days_below_0F,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,68,35,50.589286,73,40,53.693548,80,41,55.5,78,46,56.854839,84,50,60.5,90,52,62.725806,84,52,62.16129,85,52,64.65,90,47,63.016129,83,43,53.8,72,36,49.274194,71,115,2960,16.59,0.0,0,56.972603,0,0,0,0,14,0,0,0,1.0,1.0,1.0,NaN,248.682615
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,68,35,50.589286,73,40,53.693548,80,41,55.5,78,46,56.854839,84,50,60.5,90,52,62.725806,84,52,62.16129,85,52,64.65,90,47,63.016129,83,43,53.8,72,36,49.274194,71,115,2960,16.59,0.0,0,56.972603,0,0,0,0,14,0,0,0,1.0,NaN,1.0,12.0,26.500150
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,68,35,50.589286,73,40,53.693548,80,41,55.5,78,46,56.854839,84,50,60.5,90,52,62.725806,84,52,62.16129,85,52,64.65,90,47,63.016129,83,43,53.8,72,36,49.274194,71,115,2960,16.59,0.0,0,56.972603,0,0,0,0,14,0,0,0,1.0,NaN,1.0,12.0,24.693619
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,68,35,50.589286,73,40,53.693548,80,41,55.5,78,46,56.854839,84,50,60.5,90,52,62.725806,84,52,62.16129,85,52,64.65,90,47,63.016129,83,43,53.8,72,36,49.274194,71,115,2960,16.59,0.0,0,56.972603,0,0,0,0,14,0,0,0,1.0,NaN,1.0,12.0,48.406926
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,68,35,50.589286,73,40,53.693548,80,41,55.5,78,46,56.854839,84,50,60.5,90,52,62.725806,84,52,62.16129,85,52,64.65,90,47,63.016129,83,43,53.8,72,36,49.274194,71,115,2960,16.59,0.0,0,56.972603,0,0,0,0,14,0,0,0,1.0,1.0,1.0,NaN,3.899395


In [5]:
len(tr)

75757

In [6]:
len(tt)

9705

In [7]:
len(sub)

9705

In [8]:
tr.columns

Index(['Year_Factor', 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
       'january_min_temp', 'january_avg_temp', 'january_max_temp',
       'february_min_temp', 'february_avg_temp', 'february_max_temp',
       'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
       'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
       'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
       'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'september_min_temp',
       'september_avg_temp', 'september_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inc

In [9]:
len(tr.columns)

63

# Functions

In [10]:
def drop_features(df, features_to_drop):
  df = df.drop(features_to_drop, axis=1)
  return df

## Imputing Missing Values

In [11]:
def naive_impute(train, test):
  features = train.columns

  if 'year_built' in features:
    train['year_built'] = train['year_built'].replace(np.nan, -9999)
    test['year_built'] = test['year_built'].replace(np.nan, -9999)
  
  if 'energy_star_rating' in features:
    train['energy_star_rating'] = train['energy_star_rating'].replace(np.nan, -9999)
    test['energy_star_rating'] = test['energy_star_rating'].replace(np.nan, -9999)
  
  if 'direction_max_wind_speed' in features:
    train['direction_max_wind_speed']= train['direction_max_wind_speed'].replace(np.nan, train['direction_max_wind_speed'].mean())
    test['direction_max_wind_speed']= test['direction_max_wind_speed'].replace(np.nan, test['direction_max_wind_speed'].mean())
  
  if 'direction_peak_wind_speed' in features:
    train['direction_peak_wind_speed'] = train['direction_peak_wind_speed'].replace(np.nan, train['direction_peak_wind_speed'].mean())
    test['direction_peak_wind_speed'] = test['direction_peak_wind_speed'].replace(np.nan, test['direction_peak_wind_speed'].mean())
  
  if 'max_wind_speed' in features:
    train['max_wind_speed'] = train['max_wind_speed'].replace(np.nan, train['max_wind_speed'].mean())
    test['max_wind_speed'] = test['max_wind_speed'].replace(np.nan, test['max_wind_speed'].mean())
  
  if 'days_with_fog' in features:
    train['days_with_fog'] = train['days_with_fog'].replace(np.nan, train['days_with_fog'].mean())
    test['days_with_fog'] = test['days_with_fog'].replace(np.nan, test['days_with_fog'].mean())
  return train, test

## Adding Features

In [12]:
def create_interaction_feature(data, feature_name, cols):
  data[feature_name] = data[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
  return data

def add_interaction_features(data, features_toAdd):
  for feature in features_toAdd:
    data = create_interaction_feature(data, feature, features_toAdd[feature])
  return data

## Feature Transformation Functions

In [13]:
def target_encode_categs(train, test):
  categorical_features = [col for col in train.columns if train[col].dtype == 'O']
  encoder = ce.TargetEncoder(cols=categorical_features)

  X = train.drop('site_eui', axis=1)
  y = train['site_eui']

  encoder.fit(X[categorical_features], y)
  train[categorical_features] = encoder.transform(train[categorical_features])
  test[categorical_features] = encoder.transform(test[categorical_features])
  return train, test

In [14]:
def min_max_scale_features(train, test):
  categorical_features = [col for col in train.columns if train[col].dtype == 'O']
  numerical_features = [col for col in train.columns if col not in categorical_features and col != 'site_eui']
  scaler = MinMaxScaler()
  train[numerical_features] = scaler.fit_transform(train[numerical_features])
  test[numerical_features] = scaler.transform(test[numerical_features])
  return train, test

## Model Building Functions

### LightGBM

In [15]:
def train_lgbm(X_train, y_train, X_val, y_val):
  lgbm_train = lgb.Dataset(X_train, y_train)
  lgbm_val = lgb.Dataset(X_val, y_val)
  
  params = {
          'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': {'rmse'},
          "max_depth":12,
          "num_leaves":32,
          'learning_rate': 0.1,
          "min_child_samples": 100,
          'feature_fraction': 0.9,
          "bagging_freq":1,
          'bagging_fraction': 0.9,
          "lambda_l1":10,
          "lambda_l2":10,
          'min_data_per_group':500,
          'verbose': 0
          }

  model = lgb.train(params,
                    lgbm_train,
                    num_boost_round=13100,
                    valid_sets=lgbm_val,
                    early_stopping_rounds=50,
                    verbose_eval=0)
  
  return model

### Random Forest Regressor

In [16]:
def train_rf(X_train, y_train, X_val, y_val):
  model = RandomForestRegressor(random_state=42, n_estimators=150, max_depth=50)
  model.fit(X_train, y_train)
  return model

## Cross Validation Function

In [17]:
def groupkFold_cross_validate(data, impute_missing, encode_categs, scale_features, train_model):
  X = data.drop('site_eui', axis=1)
  y = data['site_eui']
  
  y_true, y_pred = list(), list()

  i = 1
  kfold = GroupKFold(n_splits=6)
  for train_idx, val_idx in kfold.split(X, y, groups=data['Year_Factor']):
    train, val = data.loc[train_idx], data.loc[val_idx]

    train, val = encode_categs(train, val)
    train, val = impute_missing(train, val)
    train, val = scale_features(train, val)
    
    X_train = train.drop('site_eui', axis=1)
    y_train = train['site_eui']

    X_val = val.drop('site_eui', axis=1)
    y_val = val['site_eui']

    model = train_model(X_train, y_train, X_val, y_val)
    
    val_pred = model.predict(X_val)

    y_pred.extend(val_pred)
    y_true.extend(y_val)

    fold_rmse = mean_squared_error(y_val, val_pred, squared=False)
    print(f'- RMSE of fold {i} = {fold_rmse}')
    i = i+1
  
  rmse = mean_squared_error(y_true, y_pred, squared=False)
  print(f'OOF RMSE = {rmse}')
  return rmse

# Preparing Data for Functions

In [18]:
train = tr.copy()
test = tt.copy()

In [19]:
features_to_drop = ['january_min_temp', 'january_max_temp',
       'february_min_temp', 'february_max_temp',
       'march_min_temp', 'march_max_temp', 'april_min_temp',
       'april_max_temp', 'may_min_temp',
       'may_max_temp', 'june_min_temp', 'june_max_temp',
       'july_min_temp', 'july_max_temp', 'august_min_temp', 'august_max_temp', 
       'september_min_temp', 'september_max_temp', 'october_min_temp',
       'october_max_temp', 'november_min_temp', 'november_max_temp', 'december_min_temp',
       'december_max_temp',
       'days_with_fog', 'direction_peak_wind_speed', 'max_wind_speed', 'direction_max_wind_speed',
        'avg_temp', 'snowfall_inches', 'heating_degree_days', 'cooling_degree_days', 
       'days_below_30F',  'days_below_10F',  'snowdepth_inches', 'building_class',
       'days_below_0F', 'days_below_20F', 'days_above_100F', 
       'august_avg_temp',  'october_avg_temp', 
       'september_avg_temp', 'november_avg_temp', 'february_avg_temp',
       'days_above_90F', 'days_above_80F', 'march_avg_temp', 'precipitation_inches',
       'january_avg_temp','june_avg_temp', 'april_avg_temp', 
        'july_avg_temp', 'may_avg_temp', 'december_avg_temp',
       'days_above_110F'
       ]

features_to_add = {
    'building_id': ['State_Factor', 'building_class', 'facility_type', 'year_built', 'floor_area', 'ELEVATION'],
    'building_id_noState': ['building_class', 'facility_type', 'year_built', 'floor_area', 'ELEVATION'],
    'building_id_noElevation': ['State_Factor', 'building_class', 'facility_type', 'year_built', 'floor_area'], # new
    'floorArea_StarRating': ['floor_area', 'energy_star_rating'],
    'floorArea_facilityType': ['floor_area', 'facility_type'],
    'floorArea_buildingClass': ['floor_area', 'building_class'],
    'floorArea_stateFactor': ['floor_area', 'State_Factor'],
    'facilityType_stateFactor': ['facility_type', 'State_Factor'],
    'yearBuilt_facilityType': ['year_built', 'facility_type'],
    'yearBuilt_floorArea': ['year_built', 'floor_area'],
    'yearBuilt_stateFactor': ['year_built', 'State_Factor'],
    'buildingClass_StarRating': ['building_class', 'energy_star_rating']
}

## Adding Features

In [20]:
train = add_interaction_features(train, features_to_add)
test = add_interaction_features(test, features_to_add)

## Dropping Features and Rows

In [21]:
train = drop_features(train, features_to_drop)
test = drop_features(test, features_to_drop)

In [22]:
train.head()

,Year_Factor,State_Factor,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,site_eui,building_id,building_id_noState,building_id_noElevation,floorArea_StarRating,floorArea_facilityType,floorArea_buildingClass,floorArea_stateFactor,facilityType_stateFactor,yearBuilt_facilityType,yearBuilt_floorArea,yearBuilt_stateFactor,buildingClass_StarRating
0,1,State_1,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,248.682615,State_1_Commercial_Grocery_store_or_food_marke...,Commercial_Grocery_store_or_food_market_1942.0...,State_1_Commercial_Grocery_store_or_food_marke...,61242.0_11.0,61242.0_Grocery_store_or_food_market,61242.0_Commercial,61242.0_State_1,Grocery_store_or_food_market_State_1,1942.0_Grocery_store_or_food_market,1942.0_61242.0,1942.0_State_1,Commercial_11.0
1,1,State_1,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,26.500150,State_1_Commercial_Warehouse_Distribution_or_S...,Commercial_Warehouse_Distribution_or_Shipping_...,State_1_Commercial_Warehouse_Distribution_or_S...,274000.0_45.0,274000.0_Warehouse_Distribution_or_Shipping_ce...,274000.0_Commercial,274000.0_State_1,Warehouse_Distribution_or_Shipping_center_State_1,1955.0_Warehouse_Distribution_or_Shipping_center,1955.0_274000.0,1955.0_State_1,Commercial_45.0
2,1,State_1,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,24.693619,State_1_Commercial_Retail_Enclosed_mall_1951.0...,Commercial_Retail_Enclosed_mall_1951.0_280025....,State_1_Commercial_Retail_Enclosed_mall_1951.0...,280025.0_97.0,280025.0_Retail_Enclosed_mall,280025.0_Commercial,280025.0_State_1,Retail_Enclosed_mall_State_1,1951.0_Retail_Enclosed_mall,1951.0_280025.0,1951.0_State_1,Commercial_97.0
3,1,State_1,Education_Other_classroom,55325.0,1980.0,46.0,1.8,48.406926,State_1_Commercial_Education_Other_classroom_1...,Commercial_Education_Other_classroom_1980.0_55...,State_1_Commercial_Education_Other_classroom_1...,55325.0_46.0,55325.0_Education_Other_classroom,55325.0_Commercial,55325.0_State_1,Education_Other_classroom_State_1,1980.0_Education_Other_classroom,1980.0_55325.0,1980.0_State_1,Commercial_46.0
4,1,State_1,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,3.899395,State_1_Commercial_Warehouse_Nonrefrigerated_1...,Commercial_Warehouse_Nonrefrigerated_1985.0_66...,State_1_Commercial_Warehouse_Nonrefrigerated_1...,66000.0_100.0,66000.0_Warehouse_Nonrefrigerated,66000.0_Commercial,66000.0_State_1,Warehouse_Nonrefrigerated_State_1,1985.0_Warehouse_Nonrefrigerated,1985.0_66000.0,1985.0_State_1,Commercial_100.0


In [23]:
test.head()

,Year_Factor,State_Factor,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,building_id,building_id_noState,building_id_noElevation,floorArea_StarRating,floorArea_facilityType,floorArea_buildingClass,floorArea_stateFactor,facilityType_stateFactor,yearBuilt_facilityType,yearBuilt_floorArea,yearBuilt_stateFactor,buildingClass_StarRating
0,7,State_1,Grocery_store_or_food_market,28484.0,1994.0,37.0,2.4,State_1_Commercial_Grocery_store_or_food_marke...,Commercial_Grocery_store_or_food_market_1994.0...,State_1_Commercial_Grocery_store_or_food_marke...,28484.0_37.0,28484.0_Grocery_store_or_food_market,28484.0_Commercial,28484.0_State_1,Grocery_store_or_food_market_State_1,1994.0_Grocery_store_or_food_market,1994.0_28484.0,1994.0_State_1,Commercial_37.0
1,7,State_1,Grocery_store_or_food_market,21906.0,1961.0,55.0,45.7,State_1_Commercial_Grocery_store_or_food_marke...,Commercial_Grocery_store_or_food_market_1961.0...,State_1_Commercial_Grocery_store_or_food_marke...,21906.0_55.0,21906.0_Grocery_store_or_food_market,21906.0_Commercial,21906.0_State_1,Grocery_store_or_food_market_State_1,1961.0_Grocery_store_or_food_market,1961.0_21906.0,1961.0_State_1,Commercial_55.0
2,7,State_1,Grocery_store_or_food_market,16138.0,1950.0,1.0,59.1,State_1_Commercial_Grocery_store_or_food_marke...,Commercial_Grocery_store_or_food_market_1950.0...,State_1_Commercial_Grocery_store_or_food_marke...,16138.0_1.0,16138.0_Grocery_store_or_food_market,16138.0_Commercial,16138.0_State_1,Grocery_store_or_food_market_State_1,1950.0_Grocery_store_or_food_market,1950.0_16138.0,1950.0_State_1,Commercial_1.0
3,7,State_1,Grocery_store_or_food_market,97422.0,1971.0,34.0,35.4,State_1_Commercial_Grocery_store_or_food_marke...,Commercial_Grocery_store_or_food_market_1971.0...,State_1_Commercial_Grocery_store_or_food_marke...,97422.0_34.0,97422.0_Grocery_store_or_food_market,97422.0_Commercial,97422.0_State_1,Grocery_store_or_food_market_State_1,1971.0_Grocery_store_or_food_market,1971.0_97422.0,1971.0_State_1,Commercial_34.0
4,7,State_1,Grocery_store_or_food_market,61242.0,1942.0,35.0,1.8,State_1_Commercial_Grocery_store_or_food_marke...,Commercial_Grocery_store_or_food_market_1942.0...,State_1_Commercial_Grocery_store_or_food_marke...,61242.0_35.0,61242.0_Grocery_store_or_food_market,61242.0_Commercial,61242.0_State_1,Grocery_store_or_food_market_State_1,1942.0_Grocery_store_or_food_market,1942.0_61242.0,1942.0_State_1,Commercial_35.0


In [24]:
len(train)

75757

In [25]:
len(test)

9705

# GroupKFold Cross Validation

In [26]:
_ = groupkFold_cross_validate(train, naive_impute, target_encode_categs, min_max_scale_features, train_lgbm)

- RMSE of fold 1 = 33.342430286183436
- RMSE of fold 2 = 26.00462532452766
- RMSE of fold 3 = 25.86087490101921
- RMSE of fold 4 = 33.44907555431845
- RMSE of fold 5 = 34.65856224420807
- RMSE of fold 6 = 45.47651733807342
OOF RMSE = 31.132537603754677


# Building Final Models

## Preparing Data for Models

In [27]:
train, test = target_encode_categs(train, test)
train, test = naive_impute(train, test)
train, test = min_max_scale_features(train, test)

X_train = train.drop('site_eui', axis=1)
y_train = train['site_eui']

## LightGBM

In [28]:
lgbm_train = lgb.Dataset(X_train, y_train)

params = {
          'task': 'train',
          'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': {'rmse'},
          "max_depth":12,
          "num_leaves":32,
          'learning_rate': 0.1,
          "min_child_samples": 100,
          'feature_fraction': 0.9,
          "bagging_freq":1,
          'bagging_fraction': 0.9,
          "lambda_l1":10,
          "lambda_l2":10,
          'min_data_per_group':500,
          'verbose': 0
          }

In [29]:
lgbm = lgb.train(params,
                  lgbm_train,
                  num_boost_round=13100)

In [30]:
lgbm_preds = lgbm.predict(test)

## Random Forest Regressor

In [31]:
rf = train_rf(X_train, y_train, None, None)

In [32]:
rf_preds = rf.predict(test)

# Making the Submission

In [38]:
ensemble_lgbm_rf = (0.5*lgbm_preds) + (0.5*rf_preds)

In [39]:
sub['site_eui'] = ensemble_lgbm_rf

In [40]:
sub['site_eui'] 

0       185.209205
1       282.222711
2       167.960621
3       234.830093
4       246.963129
           ...    
9700     22.727695
9701     21.397957
9702     35.025546
9703     27.589009
9704     18.124306
Name: site_eui, Length: 9705, dtype: float64

In [37]:
sub.to_csv("submission.csv", index = False)